## Apartado 1
Se debe implementar un sistema que permita recibir una expresión como entrada (será una expresión
formada solo por minúsculas y sin los signos de puntuación mencionados (los seis signos con los que
vamos a trabajar en este ejercicio y que serán siempre, el punto, la coma, los dos puntos, el punto y
coma, el signo de cierre de interrogación y de exclamación: `.,;:?!`), y la salida será la misma
expresión pero con los cambios correspondientes a la introducción de mayúsculas y signos de
puntuación indicados.
A un nivel alto de especificación podremos considerar que este método tiene esta signatura:
`string addPunctuationBasic(string)`

Es decir recibirá como entrada un string y devolverá como salida un string.
Como primera versión de esta función addPunctuationBasic se implementará un modelo que
simplemente cambia la primera letra por mayúscula y añade al final del string de entrada un punto.
Por ejemplo, al ejecutar

`addPunctuationBasic(“it can be a very complicated thing the ocean”)`
se obtendrá

>It can be a very complicated thing the ocean.

In [1]:
!conda install --yes --prefix {sys.prefix} numpy
import numpy as np


EnvironmentLocationNotFound: Not a conda environment: /home/germanlorenz/proyectos/MULCIA-PLN-jobs/Quizz -2/{sys.prefix}



In [2]:
# Cargamos corpus de test
with open('PunctuationTask.test.en') as f:
    PunctuationTaskTestEn = f.readlines()
# Cargamos corpus de check
with open('PunctuationTask.check.en') as f:
    PunctuationTaskCheckEn = f.readlines()


In [3]:
# Verificamos
print(PunctuationTaskTestEn[0])
print(PunctuationTaskCheckEn[0])


it can be a very complicated thing the ocean 

It can be a very complicated thing, the ocean. 



In [4]:
def addPunctuationBasic(line):
    # Convertimos el primer caracter de la oración a mayuscula
    first_letter = line[0].upper()
    # Concatenamos el caracter convertido con la oración comenzando desde el segundo caracter a n 
    # caracteR
    formatted_line = first_letter + line[1:]
    band = True
    # Tratameinto de puntuacion final operando con posibles espacios a eliminar y el retorno de carro
    if formatted_line[-1] == '\n':
        formatted_line = formatted_line[:-1]
        while band == True:
            band = False
            if formatted_line[-1] == ' ':
                formatted_line = formatted_line[:-1]
                band = True
            else:
                band = False
    formatted_line = formatted_line + '.'
    return formatted_line

In [5]:
# Probamos con un ejemplo y con el corpus de test
frase = 'vamos a probar la puntuación básica'
print(addPunctuationBasic(frase))
print(addPunctuationBasic(PunctuationTaskTestEn[3]))

Vamos a probar la puntuación básica.
We made the ocean unhappy we made people very unhappy and we made them unhealthy.


## Apartado 2
Implementar la función verifyPunctuation con la siguiente signatura
`[(pos,err)] verifyPunctuation(string check, string test)`

Para realizar esta operación se llevará a cabo una tokenización de ambos strings. En este caso se
considerarán tokens todas las secuencias de letras, números y cualquier otro signo que no sea uno de
los seis indicados como signos de puntuación en este ejercicio (`.,;:?!`)
Es decir, esta función devolverá una lista de pares, donde cada par contendrá la posición (indicada
como índice en el string check) y el tipo de error. Los errores posibles son:
* ‘I’ → Insertion
* ‘D’ → Deletion
* ‘S’ → Substitution

Por ejemplo, consideremos que el string de referencia correcto (check) es
“Hello. What’s your name?”
La tokenización generará los siguientes 6 tokens de referencia
* Token 0: Hello
* Token 1: .
* Token 2: What’s
* Token 3: your
* Token 4: name
* Token 5: ?

Consideremos que nuestro algoritmo de puntuación (un caso hipotético para analizar el algoritmo de
verificación) genera la siguiente salida
>“Hello what’s your, name?”

Es decir los tokens generados en este caso son:
* Token 0: Hello
* Token 1: what’s
* Token 2: your
* Token 3: ,
* Token 4: name
* Token 5: ?

Debemos devolver la lista de cambios necesarios para convertir la cadena de tokens generados
(hipótesis) en la cadena de tokens correctos. Para ello, podemos inspirarnos en el algoritmo de la
distancia de Levenshtein. Es importante tener en cuenta que dadas dos cadenas A y B:
> Dist(A,B) == Dist(B,A)

Por lo que podemos abordar el problema tanto desde el punto de los cambios que hay que hacer para
llegar desde la hipótesis hasta el modelo correcto, o bien desde el modelo correco (referencia o check
en la terminología de este ejercicio) hasta la hipótesis generada por nuestro algoritmo de puntuación.
Podemos ver que respecto a nuestro string de referencia (check), el string de test ha ignorado (podemos
decir que borrado respecto al de referencia un ., por tanto tendríamos el error
>(‘D’,1)

En segundo lugar, el token 2 del string de referencia (check) se ha quedado mal en el string de test ya
que en lugar de “What’s” aparece “what’s”. Se trata por tanto de un error de substitución de una palabra
por otra (en este caso por un error de introducción de mayúsculas):
>(‘S’,2)

Y en tercer lugar, entre los tokens ‘your’ y ‘name’ del string correcto (check) se ha introducido un
nuevo token en el string de test, una coma en concreto, por tanto hay un error que calificaríamos como
>(‘I’,4)

Como se puede observar, todos los errores se posicionan (usan los índices) respecto al string correcto
de referencia (check).
Así pues el resultado de

`verifyPunctuation(“Hello. What’s your name?”,`
`“Hello what’s your, name?”)`

sería

> [ (‘D’,1), (‘S’,2), (‘I’, 4) ]


In [6]:
# Funcion para tokenizar
def tokenizador(texto, tokens):
    
    # Verificacion strings con datos
    if len(texto) == 0: return 0
    texto_tokenizado = []
    
    # Generacion de tokens
    start = 0
    for i in range(len(texto)):
        # Espacios
        if texto[i] == ' ':
            texto_tokenizado.append(texto[start:i])
            start=i+1  
        # Analisis de prueba con tokens
        for j in range (len(tokens)):
            # tokens
            if texto[i] == tokens[j]:
                texto_tokenizado.append(texto[start:i])
                start=i
    return texto_tokenizado

In [7]:
# Reference explanation https://python-course.eu/applications-python/levenshtein-distance.php
def iterative_levenshtein(s, t):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
    """

    rows = len(s)+1
    cols = len(t)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings 
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = 1
            dist[row][col] = min(dist[row-1][col] + 1,      # deletion
                                 dist[row][col-1] + 1,      # insertion
                                 dist[row-1][col-1] + cost) # substitution

    return dist

In [8]:
# Re-lectura de matriz pra detectar camino optimo
def minimal_operations(matriz_levenshtein, check_len, test_len):
  
    # Generando informe
    col = test_len
    row = check_len
    problemas = []
    while col > 1:
        while row > 1:
            # print("col: "+str(col)+" fil: "+str(row)+" max fil col : "+str(check_len)+" "+str(test_len))
            # print("D: "+str(matriz_levenshtein[row-1][col])+" I: "+str(matriz_levenshtein[row][col-1])
            #       +" S: "+str(matriz_levenshtein[row-1][col-1]))
            
            
            minimo = min(matriz_levenshtein[row-1][col],    # deletion
                         matriz_levenshtein[row][col-1],   # insertion
                         matriz_levenshtein[row-1][col-1]) # substitution
            # Diagonalizo
            
            if matriz_levenshtein[row-1][col-1] == minimo:
                if matriz_levenshtein[row-1][col-1] == matriz_levenshtein[row][col] - 1:
                    # Substitution
                    problemas.append("S,"+str(row))
                col = col - 1
                row = row - 1
            elif matriz_levenshtein[row-1][col] == minimo:
                # Deletion
                problemas.append("D,"+str(row))
                row = row - 1
            else:
                # Insertion
                problemas.append("I,"+str(row))
                col = col - 1

    return np.flip(problemas)

    

In [9]:
def verifyPunctuation(check, test, tokenizar=True, extended_report=False):
    # Analizamos distancias según casos posibles combinando situaciones
    # Agregue reporte para poder visualizar la matriz en las pruebas
    # Agregue tokenizador para simplificar el procesamiento ya que el enunciado no es claro
    
    # Tokens reservados
    reserve_tokens = ['.',',',';',':','?','!']

    if tokenizar:
        # Tokenizamos input
        data_check = tokenizador(check, reserve_tokens)
        data_test = tokenizador(test, reserve_tokens)
    else:
        data_check = check
        data_test = test

    matriz_levenshtein = iterative_levenshtein(data_check, data_test)

    # Generar reporte
    if extended_report:
        # Impresion de verificacion, opcional
        print('DATOS CORRECTOS')
        print(data_check)
        print('DATOS A SER PROBADOS')
        print(data_test)
        
        # Impresion Matriz Levenshtein opcional
        rows = len(data_check)+1
        for r in range(rows):
            print(matriz_levenshtein[r])
            
    # Exportar camino
    return minimal_operations(matriz_levenshtein, len(data_check), len(data_test))

In [10]:
# Prueba de Apartado 2 tokenizada y sin tokenizar
str1 = 'Hello. What’s your name?'
str2 = 'Hello what’s your, name?'

print(verifyPunctuation(str1,str2, tokenizar = False, extended_report = True ))
print(verifyPunctuation(str1,str2, tokenizar = True, extended_report = True))
print(verifyPunctuation("Manhattan","Manahaton"))

DATOS CORRECTOS
Hello. What’s your name?
DATOS A SER PROBADOS
Hello what’s your, name?
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
[2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
[3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[7, 6, 5, 4, 3, 2, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[8, 7, 6, 5, 4, 3, 2, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[9, 8, 7, 6, 5, 4, 3, 3, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[10, 9, 8, 7, 6, 5, 4, 4, 3, 2, 3, 4, 5, 

In [11]:
# Prueba de corpus tokenizado y sin tokenizar
print(verifyPunctuation(PunctuationTaskCheckEn[3],PunctuationTaskTestEn[3]))

['D,6' 'D,12' 'D,18']


## Apartado 3

Implementar una herramienta que permita recorrer todo el corpus de test y verificación. Es decir, iría
recorriendo una a una las líneas de cada fichero (que están alineadas), aplicaría sobre la frase de test el
algoritmo básico de puntuación (apartado 1: `addPunctuationBasic()` ) y a continuación
comprobaría si el resultado es o no correcto usando la función `verifyPunctuation()` del
apartado 2.

Obtener a continuación los valores relativos a `precisión`, exhaustividad (`recall`) y `F1` para el algoritmo
`addPunctuationBasic()` implementado en el apartado 1.

Consideraremos estos valores como el baseline, el modelo más básico de puntuación que podemos
realizar para estudiar posibles mejoras.

In [16]:
!{sys.executable} -m pip install tqdm
# Progreso
from tqdm import tqdm

def analysis(check, test, precisión, recall, F1):
    
    testProceced = []
    # Obtengo frases de test procesadas con su puntuacion
    for i in tqdm(range(len(test)),ncols = 100, desc="Agrego puntuacion ...  "):
        testProceced.append(addPunctuationBasic(test[i]))
    
    test_analysis = []
    # Obtengo fidelidad del corpus de test
    for i in tqdm(range(len(test)),ncols = 100 , desc ="Verifico puntuacion ...", disable = False):
        test_analysis.append(verifyPunctuation(check[i], testProceced[i]))
        
        
    print(len(test_analysis))

zsh:1: parse error near `-m'


In [ ]:
analysis(PunctuationTaskCheckEn, PunctuationTaskTestEn, precisión=True, recall=True, F1=True)

Verifico puntuacion ...:  38%|███████████▉                   | 5516/14382 [00:02<00:03, 2887.76it/s]